### portmy database: stocks table
### stock database: buy, price tables

In [20]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
week_str = today.strftime('%Y-%U')
today, week_str

(datetime.date(2022, 1, 16), '2022-03')

In [21]:
today = today - timedelta(days=2)
wkago = today - timedelta(days=7)
today, wkago

(datetime.date(2022, 1, 14), datetime.date(2022, 1, 7))

### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [3]:
cols = 'name price_d price_w percent perform mrkt '.split()

In [15]:
format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [39]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,LPF,2022-01-14,14.00


In [40]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'LPF'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,723,LPF,SET,14.00,17.10,10.30,999.99,1.11,"23,828.13","32,721.96",9.62,0.45,724,2021-07-08 03:43:01.644228,2022-01-14 23:24:45.995239


In [41]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "LPF" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,LPF,2022-01-14,14.00,14.10,14.00,"321,360",14.00
1,LPF,2022-01-13,14.00,14.00,13.90,"202,752",13.90
2,LPF,2022-01-12,13.90,14.00,13.80,"641,183",13.90
3,LPF,2022-01-11,13.90,14.00,13.90,"322,334",14.00
4,LPF,2022-01-10,14.00,14.10,13.80,"480,493",13.80


In [42]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (today, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2022-01-14' AND name IN ('BCH', 'BGRIM', 'CKP', 'DIF', 'DOHOME', 'EPG', 'GLOBAL', 'HREIT', 'IMH', 'IP', 'IVL', 'JASIF', 'KBANK', 'KCE', 'LPF', 'MCS', 'NER', 'NOBLE', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SAT', 'SCC', 'SIS', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TOP', 'TSTH', 'TU', 'TYCN', 'WHART')
ORDER BY name, date


35

In [43]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (wkago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2022-01-07' AND name IN ('BCH', 'BGRIM', 'CKP', 'DIF', 'DOHOME', 'EPG', 'GLOBAL', 'HREIT', 'IMH', 'IP', 'IVL', 'JASIF', 'KBANK', 'KCE', 'LPF', 'MCS', 'NER', 'NOBLE', 'PTT', 'PTTGC', 'RATCH', 'RCL', 'RJH', 'SAT', 'SCC', 'SIS', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TOP', 'TSTH', 'TU', 'TYCN', 'WHART') 
ORDER BY name


35

In [44]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(35, 3)

In [45]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [46]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
21,RCL,50.50,52.75,-4.27%,Worse
0,BCH,19.90,20.50,-2.93%,Worse
9,IP,18.80,19.30,-2.59%,Worse
13,KCE,82.00,84.00,-2.38%,Worse
32,TU,20.20,20.50,-1.46%,Worse
1,BGRIM,38.75,39.25,-1.27%,Worse
22,RJH,33.00,33.25,-0.75%,Worse
30,TOP,52.25,52.25,0.00%,No Change
18,PTT,38.75,38.75,0.00%,No Change
34,WHART,12.20,12.20,0.00%,No Change


In [47]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,62.86%
Worse,20.00%
No Change,17.14%


In [48]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(359, 8)

In [49]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [50]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(35, 13)

In [51]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
13,KCE,82.00,84.00,-2.38%,Worse,SET50
32,TU,20.20,20.50,-1.46%,Worse,SET50
1,BGRIM,38.75,39.25,-1.27%,Worse,SET50
18,PTT,38.75,38.75,0.00%,No Change,SET50
30,TOP,52.25,52.25,0.00%,No Change,SET50
20,RATCH,44.50,44.25,0.56%,Better,SET50
28,TISCO,99.00,97.50,1.54%,Better,SET50
12,KBANK,145.00,142.50,1.75%,Better,SET50
24,SCC,389.00,380.00,2.37%,Better,SET50
19,PTTGC,60.50,59.00,2.54%,Better,SET50


In [52]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,58.33%
Worse,25.00%
No Change,16.67%


In [53]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
21,RCL,50.50,52.75,-4.27%,Worse,SET100
0,BCH,19.90,20.50,-2.93%,Worse,SET100
2,CKP,5.00,5.00,0.00%,No Change,SET100
4,DOHOME,22.70,22.70,0.00%,No Change,SET100
5,EPG,12.30,12.20,0.82%,Better,SET100
26,STA,31.50,30.75,2.44%,Better,SET100
27,SYNEX,34.00,32.25,5.43%,Better,SET100


In [54]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,42.86%
Worse,28.57%
No Change,28.57%


In [55]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
22,RJH,33.00,33.25,-0.75%,Worse,SET999
7,HREIT,8.95,8.95,0.00%,No Change,SET999
34,WHART,12.20,12.20,0.00%,No Change,SET999
15,MCS,14.90,14.80,0.68%,Better,SET999
29,TMT,10.30,10.20,0.98%,Better,SET999
3,DIF,14.10,13.90,1.44%,Better,SET999
14,LPF,14.00,13.80,1.45%,Better,SET999
33,TYCN,3.66,3.60,1.67%,Better,SET999
11,JASIF,10.60,10.40,1.92%,Better,SET999
25,SIS,45.50,44.50,2.25%,Better,SET999


In [56]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,78.57%
No Change,14.29%
Worse,7.14%


In [57]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
9,IP,18.80,19.30,-2.59%,Worse,mai
8,IMH,19.80,16.40,20.73%,Better,mai


In [58]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,50.00%
Worse,50.00%
